In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time
import ta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data.get_data import *

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import ta

from indicator import *
from multianalysis.volatility import GARCH

start = "2023"
end = "2023"

data = Get_data("BTC", "1d")
data = data.loc[start : end]

rets = data.close.pct_change().dropna()*100
rets.name = "returns"

ma_1 = 3
ma_2 = 7
ma_3 = 14

atr_ = 24
rsi_ = 14

window = 10

data['returns'] = data.close.pct_change()

data['ma_1'] = ma(data, ma_1)
data['ma_2'] = ma(data, ma_2)
data['ma_3'] = ma(data, ma_3)

data['vol_std'] = rets.rolling(window).std()
data['vol'] = rets.ewm(span = 5).std()

data['sar'], data['sar_down'], data['sar_up'] = sar(data)

data['garch'] = GARCH(rets)

data['r_target'] = data['returns'].rolling(3).mean()
data['target'] = np.where(data['r_target'] > 0, 1, 0)

                       Zero Mean - GARCH Model Results                        
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.005
Vol Model:                      GARCH   Log-Likelihood:               -468.739
Distribution:                  Normal   AIC:                           943.478
Method:            Maximum Likelihood   BIC:                           953.418
                                        No. Observations:                  203
Date:                Sat, Aug 12 2023   Df Residuals:                      203
Time:                        04:03:54   Df Model:                            0
                            Volatility Model                            
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          4.0209      1.023      3.931  8.467e-05 [  2.016,  6.02

In [6]:
from utils.plot import *

fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.8, 0.2], vertical_spacing = 0.01)
#shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]
fig.add_trace(
    go.Candlestick(
        x = data.index , open = data.open, close = data.close,
        high = data.high, low = data.low, name = data.symbol[0],
        yaxis='y1'
    ),
    col = 1, row = 1
)


add_line(fig, x=data.index, y=data['ma_1'], name='ma_1', color='blue')
add_line(fig, x=data.index, y=data['ma_2'], name='ma_2', color='orange')
add_line(fig, x=data.index, y=data['ma_3'], name='ma_3', color='black')
add_mark(fig, x=data.index, y=data['sar_up'], name='sar_up', color='blue')
add_mark(fig, x=data.index, y=data['sar_down'], name='sar_down', color='black')


add_line(fig, x=data.index, y=data['target'], name='target', color='blue', row = 2 , col = 1)
add_line(fig, x=data.index, y=data['vol'], name='vol', color='blue', row = 2 , col = 1)
add_line(fig, x=data.index, y=data['garch'], name='garch', color='green', row = 2 , col = 1)



fig.update_xaxes(rangeslider_visible = False , col =1, row =1)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

#Fig = go.FigureWidget(fig)
fig.update_layout(height = 800 , width =1500,
                  margin=dict(
                        l=5, r=0, b=10, t=30,
                        pad=1
                    ),
                  showlegend=True,
                  )
fig.show()

## Optimiser Triple MM

In [9]:
from sklearn.model_selection import ParameterGrid


In [16]:
MM = {
    'a_low' : [2, 4, 6, 8],
    'b_meduim' : [9, 11, 13, 15],
    'c_high' : [16, 18, 20, 22]
}

list(ParameterGrid(MM))

[{'a_low': 2, 'b_meduim': 9, 'c_high': 16},
 {'a_low': 2, 'b_meduim': 9, 'c_high': 18},
 {'a_low': 2, 'b_meduim': 9, 'c_high': 20},
 {'a_low': 2, 'b_meduim': 9, 'c_high': 22},
 {'a_low': 2, 'b_meduim': 11, 'c_high': 16},
 {'a_low': 2, 'b_meduim': 11, 'c_high': 18},
 {'a_low': 2, 'b_meduim': 11, 'c_high': 20},
 {'a_low': 2, 'b_meduim': 11, 'c_high': 22},
 {'a_low': 2, 'b_meduim': 13, 'c_high': 16},
 {'a_low': 2, 'b_meduim': 13, 'c_high': 18},
 {'a_low': 2, 'b_meduim': 13, 'c_high': 20},
 {'a_low': 2, 'b_meduim': 13, 'c_high': 22},
 {'a_low': 2, 'b_meduim': 15, 'c_high': 16},
 {'a_low': 2, 'b_meduim': 15, 'c_high': 18},
 {'a_low': 2, 'b_meduim': 15, 'c_high': 20},
 {'a_low': 2, 'b_meduim': 15, 'c_high': 22},
 {'a_low': 4, 'b_meduim': 9, 'c_high': 16},
 {'a_low': 4, 'b_meduim': 9, 'c_high': 18},
 {'a_low': 4, 'b_meduim': 9, 'c_high': 20},
 {'a_low': 4, 'b_meduim': 9, 'c_high': 22},
 {'a_low': 4, 'b_meduim': 11, 'c_high': 16},
 {'a_low': 4, 'b_meduim': 11, 'c_high': 18},
 {'a_low': 4, 'b_m

# Backtest

In [2]:
from backtest.app import App

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [3]:
SYMBOLS = ["BTC"]
CAPITAL = 100
START = "2022-01"
E = "2023-06"

app = App(symbols=SYMBOLS, capital=CAPITAL, start=START, end=E)
app.run(report = False)

546 / 546


In [6]:
for params in portfolios:
    app = App(symbols=SYMBOLS, capital=CAPITAL, start=START, end=E)
    app.run(report = False)

546 / 546


In [ ]:
app = App(symbols=SYMBOLS, capital=CAPITAL)
app.run(report = True)

204 / 204


In [ ]:
=n

In [5]:
from simulation import simule
from multiprocessing import Process



portfolios = [
        {"symbols": ["ETH"], "capital": 10000, "start": "2022-01", "end": "2023-06", "interval": "1d", "db_trades": "sim1"},
        {"symbols": ["XMR"], "capital": 10000, "start": "2022-01", "end": "2023-06", "interval": "1d", "db_trades": "sim2"},
        {"symbols": ["BTC"], "capital": 10000, "start": "2022-01", "end": "2023-06", "interval": "1d", "db_trades": "sim3"},
        {"symbols": ["ETH"], "capital": 10000, "start": "2022-01", "end": "2023-06", "interval": "1d", "db_trades": "sim4"}
    ]



processes = []
for params in portfolios:
    p = Process(target = simule, args=(params,))
    processes.append(p)
    p.start()
            
for p in processes:
    p.join()

print("All simulations are completed.")

All simulations are completed.
